In [3]:
# 
import json
import cv2
import requests
from tencentcloud.common import credential
from tencentcloud.common.profile.client_profile import ClientProfile
from tencentcloud.common.profile.http_profile import HttpProfile
from tencentcloud.common.exception.tencent_cloud_sdk_exception import TencentCloudSDKException
from tencentcloud.bda.v20200324 import bda_client
from tencentcloud.bda.v20200324 import models as bda_models

import numpy as np
import base64
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
with open("../tencent_key.json" ,'r') as key:
    SecretDict=json.loads(key.read())
SecretId=SecretDict["SecretId"]
SecretKey=SecretDict["SecretKey"]

In [4]:
"""
视频人像分割
https://console.cloud.tencent.com/api
/explorer?Product=bda&Version=2020-03-24&Action=CreateSegmentationTask&SignVersion=
"""
try: 
    cred = credential.Credential(SecretId,SecretKey) 
    httpProfile = HttpProfile()
    httpProfile.endpoint = "bda.tencentcloudapi.com"

    clientProfile = ClientProfile()
    clientProfile.httpProfile = httpProfile
    client = bda_client.BdaClient(cred, "ap-guangzhou", clientProfile) 

    req = bda_models.CreateSegmentationTaskRequest()
    """
    VideoUrl	是	String	需要分割的视频URL，可外网访问。
    BackgroundImageUrl	否	String	背景图片URL。
    可以将视频背景替换为输入的图片。
    如果不输入背景图片，则输出人像区域mask。
    Config	否	String	预留字段，后期用于展示更多识别信息。
    """
    params = {
        "VideoUrl": "https://webplus-cn-beijing-s-5f12835a3dd7f938e565a84d.oss-cn-beijing.aliyuncs.com/video/test_10s.mp4",
        "BackgroundImageUrl": "",
        "Config": ""
    }
    req.from_json_string(json.dumps(params))

    resp = client.CreateSegmentationTask(req).to_json_string() 
    resp=json.loads(resp)
    print(resp) 

except TencentCloudSDKException as err: 
    print(err) 

{'TaskID': '9YZqUUM2nzElnQxt', 'EstimatedProcessingTime': 30, 'RequestId': 'efba7af0-a341-40c5-800d-c35167a1fb9d'}


In [5]:
TaskID=resp["TaskID"]
TaskID

'9YZqUUM2nzElnQxt'

In [7]:

try: 
    cred = credential.Credential(SecretId, SecretKey) 
    httpProfile = HttpProfile()
    httpProfile.endpoint = "bda.tencentcloudapi.com"

    clientProfile = ClientProfile()
    clientProfile.httpProfile = httpProfile
    client = bda_client.BdaClient(cred, "ap-guangzhou", clientProfile) 

    req = bda_models.DescribeSegmentationTaskRequest()
    params = {
        "TaskID": TaskID
    }
    req.from_json_string(json.dumps(params))

    resp = client.DescribeSegmentationTask(req).to_json_string()
    resp=json.loads(resp)
    print(resp) 

except TencentCloudSDKException as err: 
    print(err) 

{'TaskStatus': 'FINISHED', 'ResultVideoUrl': 'http://bda-video-bodyseg-1254418846.cos.ap-guangzhou.myqcloud.com/video_segment_prod/v1.0/1304700566/20210820105702_efba7af0-a341-40c5-800d-c35167a1fb9d_result.mp4', 'ResultVideoMD5': 'ECE260F2BF662BC162FD6DB5E142A84D', 'VideoBasicInformation': {'FrameWidth': 1920, 'FrameHeight': 1080, 'FramesPerSecond': 25, 'Duration': 10, 'TotalFrames': 250}, 'ErrorMsg': '', 'RequestId': 'd3423ad7-3fac-4cf1-a214-8024d071ca10'}


In [9]:
ResultVideoUrl=resp['ResultVideoUrl']
ResultVideoUrl

'http://bda-video-bodyseg-1254418846.cos.ap-guangzhou.myqcloud.com/video_segment_prod/v1.0/1304700566/20210820105702_efba7af0-a341-40c5-800d-c35167a1fb9d_result.mp4'

In [10]:
with open("./video/视频人像分割.mp4",'wb') as file:
    file.write(requests.get(ResultVideoUrl).contenttent)